In [2]:
import requests
from get_api_keys import API_KEY_FINNHUB
from datetime import datetime, timedelta
import os
import pandas as pd
import json
import finnhub

FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Ynov\\M2\\Projet Master\\Projet_bourse\\Projet_master\\SRC\\API\\API_KEYS.json'

# <h1 style="color:red"> API FINHUB

In [3]:
finnhub_client = finnhub.Client(api_key=API_KEY_FINNHUB)

## <h2 style="color:purple"> NEWS

### <h3 style="color:cyan"> get_company_news

In [4]:
def get_company_news(business_name, begin_date, end_date):
    news = finnhub_client.company_news(business_name, _from=begin_date, to=end_date)
    news = pd.json_normalize(news)
    news["datetime"] = news["datetime"].apply(lambda x : datetime.fromtimestamp(x))
    news["year"] = news["datetime"].apply(lambda x : x.year)
    news["month"] = news["datetime"].apply(lambda x : x.month)
    news["day"] = news["datetime"].apply(lambda x : x.day)
    print(f"News sur {news.loc[0, "datetime"] - news.loc[len(news)-1, "datetime"]} jours ")
    return news

In [5]:
from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
today = datetime.now().strftime('%Y-%m-%d')

#a = get_company_news('TSLA', "2024-01-01", "2024-11-01")
finnhub_business_news = get_company_news('TSLA', from_date, today)

News sur 7 days 05:29:11 jours 


In [6]:
finnhub_business_news.shape

(212, 12)

In [7]:
finnhub_business_news.head(3)

,category,datetime,headline,id,image,related,source,summary,url,year,month,day
0,company,2024-12-14 07:43:10,What Moved Markets This Week,131982553,https://static.seekingalpha.com/cdn/s3/uploads...,TSLA,SeekingAlpha,The awful market breadth and concerns sparked ...,https://finnhub.io/api/news?id=591a3c242f8ff56...,2024,12,14
1,company,2024-12-14 00:54:16,"Best Way To Invest $100,000: Build An 8-Stock ...",131977467,https://static.seekingalpha.com/cdn/s3/uploads...,TSLA,SeekingAlpha,Learn how to maximize returns by focusing on h...,https://finnhub.io/api/news?id=55d1529df95faa2...,2024,12,14
2,company,2024-12-13 23:55:34,Tesla raises prices of Model S cars in US by $...,131976825,,TSLA,Finnhub,Tesla on Friday raised the prices of its Model...,https://finnhub.io/api/news?id=651b5a9382b6a7c...,2024,12,13


### <h3 style="color:cyan"> get_general_news_country

In [8]:
def get_general_news_country():
    news = pd.json_normalize(
        finnhub_client.general_news('general') # Accès aux dernières nouvelles générales, de forex, de crypto, et de fusions.
    )
    news["datetime"] = news["datetime"].apply(lambda x : datetime.fromtimestamp(x))
    news["year"] = news["datetime"].apply(lambda x : x.year)
    news["month"] = news["datetime"].apply(lambda x : x.month)
    news["day"] = news["datetime"].apply(lambda x : x.day)
    print(f"News sur {news.loc[0, "datetime"] - news.loc[len(news)-1, "datetime"]} jours ")
    return news


In [9]:
finnhub_country_news = get_general_news_country()

News sur 1 days 21:55:32 jours 


In [10]:
finnhub_country_news.shape

(100, 12)

In [11]:
finnhub_country_news.head(3)

,category,datetime,headline,id,image,related,source,summary,url,year,month,day
0,top news,2024-12-14 02:09:09,Bitcoin proxy MicroStrategy to join the Nasdaq...,7434396,https://image.cnbcfm.com/api/v1/image/10691965...,,CNBC,"MicroStrategy, the high beta play on the price...",https://www.cnbc.com/2024/12/13/bitcoin-proxy-...,2024,12,14
1,top news,2024-12-14 00:33:26,Cramer's week ahead: Fed decision and earnings...,7434393,https://image.cnbcfm.com/api/v1/image/10735022...,,CNBC,CNBC's Jim Cramer walked investors through nex...,https://www.cnbc.com/2024/12/13/cramers-week-a...,2024,12,14
2,top news,2024-12-13 22:25:11,Tech companies most threatened by Trump are do...,7434380,https://image.cnbcfm.com/api/v1/image/10807285...,,CNBC,"Amazon, Meta and OpenAI CEO Sam Altman have al...",https://www.cnbc.com/2024/12/13/tech-companies...,2024,12,13


## <h2 style="color:purple"> Financial information about the company

### <h3 style="color:cyan"> Competitor

In [12]:
def get_competitor(business_name):
    """ get concurent"""
    company_peers = finnhub_client.company_peers(business_name)
    return company_peers

In [ ]:
get_competitor('AAPL')

### <h3 style="color:cyan"> Recommendation trends

In [ ]:
def recommendation_trends(business_name):
    # Recommendation trends Recommandations des analystes, cibles de prix et surprises sur les bénéfices.
    recommendation = pd.json_normalize(finnhub_client.recommendation_trends(business_name))
    return recommendation

In [ ]:
recommendation_trends("AAPL")

### <h3 style="color:cyan"> Insider sentiment

In [17]:
def insider_sentiment(business_name, begin_date, end_date):
    """
    sentiment des initiés (dirigeants d'entreprises) concernant des sociétés américaines. Cette mesure est calculée en utilisant la méthode MSPR (Mean Sentiment Per Report), 
    qui évalue le sentiment global exprimé dans les rapports d'insiders.
    L'échelle de MSPR va de -100 (très négatif) à 100 (très positif)
    Ces sentiments peuvent être utilisés pour anticiper des changements de prix dans les 30 à 90 jours à venir.
    """
    df = pd.json_normalize(finnhub_client.stock_insider_sentiment(business_name, begin_date, end_date)["data"])
    return df

In [18]:
insider_sentiment('AAPL', from_date, today)

,symbol,year,month,change,mspr
0,AAPL,2024,2,-89388,-63.737488
1,AAPL,2024,4,-652766,-33.164017
2,AAPL,2024,5,-81849,-100.000000
3,AAPL,2024,8,-118384,-100.000000
4,AAPL,2024,9,549640,100.000000
5,AAPL,2024,10,-990414,-33.240925
6,AAPL,2024,11,-212130,-100.000000


### <h3 style="color:cyan"> Info businness

In [28]:
def info_company(business_name):
    df = pd.json_normalize(finnhub_client.company_profile2(symbol=business_name))
    df = df[["country", "currency", "finnhubIndustry", "marketCapitalization"]]
    return df 

In [29]:
info_company("AAPL")

,country,currency,finnhubIndustry,marketCapitalization
0,US,USD,Technology,3.750689e+06
